<a href="https://colab.research.google.com/github/Sehastrajit-clg/Deep-Learning/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [38]:
import torch
from torch import nn

In [39]:
def dropout_layer(x,dropout):
  assert 0<= dropout <= 1
  if dropout==1: return torch.zeros_like(x)
  mask=(torch.rand(X.shape)>dropout).float()
  #print("mask",mask)
  return mask*X/(1.0-dropout)

In [40]:
X=torch.arange(16,dtype=torch.float32).reshape((2,8))
print(X)
print('dropout_p=0:',dropout_layer(X,0))
print('dropout_p=0.5:',dropout_layer(X,0.5))
print('dropout_p=1:',dropout_layer(X,1))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
dropout_p=0: tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
dropout_p=0.5: tensor([[ 0.,  0.,  4.,  0.,  0.,  0., 12., 14.],
        [16., 18., 20.,  0., 24.,  0.,  0.,  0.]])
dropout_p=1: tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [41]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [42]:
#read Data
data=pd.read_csv("sonar_data.csv")
X=data.iloc[:,1:61]
y=data.iloc[:,61]

encoder=LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)


X=torch.tensor(X.values,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.float32).reshape(-1,1)
#print(x)

In [53]:
class SonarModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(60,60),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(60,30),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(30,1),
        nn.Sigmoid()

    )
  def forward(self,x):
    probas = self.layers(x)
    return probas

In [54]:
def model_train(model, X_train,y_train,X_val,y_val,n_epochs=200,batch_size=16):
  loss_fn = nn.BCELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  batch_start = torch.arange(0, len(X_train), batch_size)

  model.train()
  for epoch in range(n_epochs):
      for start in batch_start:
          X_batch = X_train[start:start+batch_size]
          y_batch = y_train[start:start+batch_size]
          y_pred = model(X_batch)
          loss = loss_fn(y_pred, y_batch)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
  model.eval()
  y_pred = model(X_val)
  acc = (y_pred.round() == y_val).float().mean()
  acc = float(acc)
  return acc


In [56]:
import numpy as np

In [58]:
kfold = StratifiedKFold(n_splits=10,shuffle=True)
accuracies=[]
for train,test in kfold.split(X,y):
  model=SonarModel()
  acc=model_train(model,X[train],y[train],X[test],y[test])
  print("Accuracy: %.2f"%acc)
  accuracies.append(acc)

Accuracy: 0.90
Accuracy: 0.95
Accuracy: 0.95
Accuracy: 0.86
Accuracy: 0.81
Accuracy: 0.71
Accuracy: 0.76
Accuracy: 0.86
Accuracy: 0.90
Accuracy: 0.85


In [59]:
mean = np.mean(accuracies)
std = np.std(accuracies)
print("Baseline: %.2f%% (+/- %.2f%%)" % (mean*100, std*100))

Baseline: 85.60% (+/- 7.35%)
